In [1]:
import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms.transforms import ToTensor

from models.model import My_siamese
from utils.utils import increment_path
from pathlib import Path

from tqdm import tqdm
import wandb

import numpy as np
import matplotlib.pyplot as plt

from dataset.reconstruct import prepare_data
from dataset.loader import train_validation_loader
from dataset.loader import testset_loader

In [2]:

net = My_siamese().cuda()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

data_dir = os.path.join("data")
datasets.Omniglot(data_dir, background=True, download=True, transform=ToTensor())
datasets.Omniglot(data_dir, background=False, download=True, transform=ToTensor())


Files already downloaded and verified
Files already downloaded and verified


Dataset Omniglot
    Number of datapoints: 0
    Root location: data/omniglot-py
    StandardTransform
Transform: ToTensor()

In [3]:
train_dir, val_dir, test_dir = prepare_data(data_dir)
train_loader, val_loader = train_validation_loader(train_dir, val_dir, 8, False, 5, True, 3, 8)
   

In [5]:
net.train()
train_loss = 0

for i, (img1, img2, label) in tqdm(enumerate(train_loader), total=len(train_loader)):

    output = net(img1.cuda(), img2.cuda())
    loss = loss_fn(output, label.cuda())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss

    if (i+1) % (len(train_loader)//5) == 0 or i == (len(train_loader) -1):
        # wandb.log({"Train_loss":train_loss/i})
        print("Train_loss", train_loss/i})

train_loss /= len(train_loader)


'data'